In [8]:
import torch
import torch.nn as nn 
import torch.optim as optim 


In [9]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [10]:
import pandas as pd 
from sklearn.preprocessing import StandardScaler 



data = pd.read_csv("custom_sample_mass_radius_k2_15.csv", 
                   header=None, 
                   names=["Mass", "Radius","k2", "Type"], 
                   low_memory=False)

m_r_p = data.iloc[1:, :-1]
star_type = data.iloc[1:, -1]

scaler = StandardScaler()
m_r_p = scaler.fit_transform(m_r_p)



In [59]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report, confusion_matrix

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

# 1. Load dataset
df = pd.read_csv("custom_sample_mass_radius_k2_15.csv", 
                 )

# 2. Separate features and target
X = df.drop(columns=["Type"]).values.astype(np.float32)
y = df["Type"].values

# 3. Encode labels
encoder = LabelEncoder()
y_encoded = encoder.fit_transform(y)

# 4. Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size=0.4, random_state=42, stratify=y_encoded
)

# 5. Scale features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# 6. Convert to tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# 7. Dataloaders
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# 8. Define Neural Network
class Net(nn.Module):
    def __init__(self, input_dim, hidden1=14, hidden2=7, output_dim= len(np.unique(y))):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden1)
        self.fc2 = nn.Linear(hidden1, hidden2)
        self.fc3 = nn.Linear(hidden2, output_dim)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

input_dim = X_train.shape[1]
output_dim = len(np.unique(y_train))
model = Net(input_dim=input_dim, output_dim=output_dim)

# 9. Loss & optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 10. Training loop
epochs = 50
for epoch in range(epochs):
    model.train()
    running_loss = 0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    if (epoch+1) % 10 == 0:
        print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader):.4f}")

# 11. Evaluation
model.eval()
y_pred = []
with torch.no_grad():
    for inputs, _ in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        y_pred.extend(predicted.numpy())




Epoch 10/50, Loss: 0.0000
Epoch 20/50, Loss: 0.0000
Epoch 30/50, Loss: 0.0000
Epoch 40/50, Loss: 0.0000
Epoch 50/50, Loss: 0.0000


In [17]:
import torch 
import torch.nn as nn
import torch.nn.functional as F


class Model(nn.Module):
    def __init__(self, in_features=3, h1=8, h2=9, out_features=1):
        super().__init__()
        self.fc1 = nn.Linear(in_features,h1)
        self.fc2 = nn.Linear(h1,h2)
        self.out = nn.Linear(h2, out_features)
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.out(x)
        return x
        

In [34]:
torch.manual_seed(42)
model = Model()
# 6. Convert to tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)


In [42]:
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Μετατροπή target
y_train_tensor = y_train_tensor.view(-1, 1).float()

epochs = 300

losses = []

for i in range(epochs):
    y_pred = model(X_train_tensor)  # (N,1)
    loss = criterion(y_pred, y_train_tensor)  # target (N,1), float
    losses.append(loss.item())
    if i % 10 == 0 :print(f"Epoch: {i+1} | Loss: {loss.item():.4f}")
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()


Epoch: 1 | Loss: 0.6356
Epoch: 11 | Loss: 0.6235
Epoch: 21 | Loss: 0.6100
Epoch: 31 | Loss: 0.5948
Epoch: 41 | Loss: 0.5777
Epoch: 51 | Loss: 0.5584
Epoch: 61 | Loss: 0.5374
Epoch: 71 | Loss: 0.5150
Epoch: 81 | Loss: 0.4916
Epoch: 91 | Loss: 0.4673
Epoch: 101 | Loss: 0.4423
Epoch: 111 | Loss: 0.4165
Epoch: 121 | Loss: 0.3906
Epoch: 131 | Loss: 0.3647
Epoch: 141 | Loss: 0.3397
Epoch: 151 | Loss: 0.3156
Epoch: 161 | Loss: 0.2928
Epoch: 171 | Loss: 0.2713
Epoch: 181 | Loss: 0.2512
Epoch: 191 | Loss: 0.2325
Epoch: 201 | Loss: 0.2152
Epoch: 211 | Loss: 0.1993
Epoch: 221 | Loss: 0.1848
Epoch: 231 | Loss: 0.1715
Epoch: 241 | Loss: 0.1595
Epoch: 251 | Loss: 0.1485
Epoch: 261 | Loss: 0.1386
Epoch: 271 | Loss: 0.1295
Epoch: 281 | Loss: 0.1213
Epoch: 291 | Loss: 0.1138


In [44]:
with torch.no_grad():
    y_eval = model(X_test_tensor)
    loss = criterion(y_eval, y_test_tensor.view(-1, 1).float())
    
    
    

In [45]:
loss

tensor(0.1063)

In [48]:
correct = 0 
with torch.no_grad():
    for i , data in enumerate(X_test_tensor):
        y_val = model.forward(data)
        if y_val.argmax().item() == y_test_tensor[i]:
            correct += 1
print(f"Test accuracy: {correct}")

Test accuracy: 12622


In [49]:
print(len(y_test_tensor))

25608
